# 1. Import libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import ETFs as etfs

Total number of ETFs: 33


# 2. Read assets from csv file

In [2]:
data = pd.read_csv('assets_final.csv', parse_dates=['Date'])
# print(data.columns.tolist())
data.set_index('Date', inplace=True)
display(data)


,Ticker,Close,High,Low,Open,Volume
Date,,,,,,
2012-01-03,CORN,42.619999,42.990002,42.180000,42.730000,114100.0
2012-01-03,DBO,26.042212,26.094875,25.770116,25.796447,318300.0
2012-01-03,EWA,12.307327,12.368641,12.234866,12.240439,2787500.0
2012-01-03,EWC,20.625568,20.648152,20.384686,20.437379,6790400.0
2012-01-03,EWG,14.720881,14.838238,14.706212,14.742886,3892700.0
...,...,...,...,...,...,...
2025-03-28,VDNR.L,136.095001,138.505005,136.065002,137.910004,31820.0
2025-03-28,VGK,70.980003,71.260002,70.820000,71.190002,6081100.0
2025-03-28,VNQ,89.709999,90.279999,89.129997,90.120003,3602900.0


# 3. Pick Closing prices 

In [3]:
data_close = data[['Ticker', 'Close']].copy()
display(data_close)
# data_close[data_close['Ticker'] == 'SPY']['Close'].plot()

,Ticker,Close
Date,,
2012-01-03,CORN,42.619999
2012-01-03,DBO,26.042212
2012-01-03,EWA,12.307327
2012-01-03,EWC,20.625568
2012-01-03,EWG,14.720881
...,...,...
2025-03-28,VDNR.L,136.095001
2025-03-28,VGK,70.980003
2025-03-28,VNQ,89.709999


# 4. Pivot the dataframe as the dates as the row, and columns as the ticker's name

In [4]:
close_wide = data_close.pivot(columns='Ticker', values='Close')
close_wide = close_wide.fillna(-1)
display(close_wide)

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,-1.000000,-1.000,-1.000000,42.619999,26.042212,-1.000000,-1.000000,12.307327,20.625568,14.720881,...,28.830000,100.450691,84.200859,317.519989,69.905647,-1.000000,27.664726,34.591934,33.707794,6.78375
2012-01-04,-1.000000,-1.000,-1.000000,42.509998,26.165094,-1.000000,-1.000000,12.452250,20.587934,14.581522,...,28.389999,100.608231,83.199745,318.160004,70.114937,-1.000000,27.328682,34.000534,33.888035,6.69750
2012-01-05,-1.000000,-1.000,-1.000000,41.509998,25.814003,-1.000000,-1.000000,12.290603,20.459965,14.412819,...,28.510000,100.876106,83.051743,313.359985,69.817871,-1.000000,26.831091,34.319893,33.472141,6.68250
2012-01-06,-1.000000,-1.000,-1.000000,41.349998,25.919333,-1.000000,-1.000000,12.128960,20.249189,14.148771,...,27.910000,100.616119,83.707436,313.760010,69.392525,-1.000000,26.462751,34.207516,33.070114,6.72625
2012-01-09,-1.000000,-1.000,-1.000000,42.189999,25.831560,-1.000000,-1.000000,12.167979,20.309414,14.207447,...,28.129999,100.860359,83.559319,312.640015,69.655838,-1.000000,26.663076,34.089245,33.097847,6.69625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,38.276001,10.405,100.550003,18.930000,14.140000,107.930000,56.490002,23.700001,41.509998,38.029999,...,29.959999,574.080017,89.120590,74.879997,129.030991,140.270004,71.730003,90.728004,74.690002,8.28750
2025-03-25,38.380001,10.440,100.720001,18.770000,14.030000,107.330002,56.580002,23.809999,41.709999,38.380001,...,30.580000,575.460022,89.110664,74.820000,129.460007,140.479996,72.160004,89.709999,75.089996,8.14875
2025-03-26,37.854000,10.400,98.169998,18.559999,14.110000,105.839996,56.080002,23.709999,41.380001,37.560001,...,30.530001,568.590027,88.524925,75.419998,130.250000,139.595001,71.139999,90.059998,74.410004,8.14500


# 5. Check and drop NaN

In [17]:
used_data = close_wide.copy()
used_data = used_data.dropna()
display(used_data)
used_data['AIEQ']
print(used_data['AIEQ'].isna().sum())

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,-1.000000,-1.000,-1.000000,42.619999,26.042212,-1.000000,-1.000000,12.307327,20.625568,14.720881,...,28.830000,100.450691,84.200859,317.519989,69.905647,-1.000000,27.664726,34.591934,33.707794,6.78375
2012-01-04,-1.000000,-1.000,-1.000000,42.509998,26.165094,-1.000000,-1.000000,12.452250,20.587934,14.581522,...,28.389999,100.608231,83.199745,318.160004,70.114937,-1.000000,27.328682,34.000534,33.888035,6.69750
2012-01-05,-1.000000,-1.000,-1.000000,41.509998,25.814003,-1.000000,-1.000000,12.290603,20.459965,14.412819,...,28.510000,100.876106,83.051743,313.359985,69.817871,-1.000000,26.831091,34.319893,33.472141,6.68250
2012-01-06,-1.000000,-1.000,-1.000000,41.349998,25.919333,-1.000000,-1.000000,12.128960,20.249189,14.148771,...,27.910000,100.616119,83.707436,313.760010,69.392525,-1.000000,26.462751,34.207516,33.070114,6.72625
2012-01-09,-1.000000,-1.000,-1.000000,42.189999,25.831560,-1.000000,-1.000000,12.167979,20.309414,14.207447,...,28.129999,100.860359,83.559319,312.640015,69.655838,-1.000000,26.663076,34.089245,33.097847,6.69625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,38.276001,10.405,100.550003,18.930000,14.140000,107.930000,56.490002,23.700001,41.509998,38.029999,...,29.959999,574.080017,89.120590,74.879997,129.030991,140.270004,71.730003,90.728004,74.690002,8.28750
2025-03-25,38.380001,10.440,100.720001,18.770000,14.030000,107.330002,56.580002,23.809999,41.709999,38.380001,...,30.580000,575.460022,89.110664,74.820000,129.460007,140.479996,72.160004,89.709999,75.089996,8.14875
2025-03-26,37.854000,10.400,98.169998,18.559999,14.110000,105.839996,56.080002,23.709999,41.380001,37.560001,...,30.530001,568.590027,88.524925,75.419998,130.250000,139.595001,71.139999,90.059998,74.410004,8.14500


0


# 6. Normalize data by the first price

In [18]:
first_prices = used_data.apply(
    lambda col: col[col != -1].iloc[0] if (col != -1).any() else np.nan
)
fp_df = pd.DataFrame(first_prices.values, index=used_data.columns, columns=['First Price'])
display(fp_df.T)

normalized_close = used_data / first_prices

normalized_close = normalized_close.mask(used_data < 0, -1) 
normalized_array = normalized_close.to_numpy(dtype='float32')

display(normalized_close)

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
First Price,21.902437,6.246624,40.0,42.619999,26.042212,22.380075,42.757004,12.307327,20.625568,14.720881,...,28.83,100.450691,84.200859,317.519989,69.905647,42.300953,27.664726,34.591934,33.707794,6.78375


Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,-1.000000,-1.000000,-1.000000,1.000000,1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000
2012-01-04,-1.000000,-1.000000,-1.000000,0.997419,1.004719,-1.000000,-1.000000,1.011775,0.998175,0.990533,...,0.984738,1.001568,0.988110,1.002016,1.002994,-1.000000,0.987853,0.982904,1.005347,0.987286
2012-01-05,-1.000000,-1.000000,-1.000000,0.973956,0.991237,-1.000000,-1.000000,0.998641,0.991971,0.979073,...,0.988900,1.004235,0.986353,0.986898,0.998744,-1.000000,0.969866,0.992136,0.993009,0.985075
2012-01-06,-1.000000,-1.000000,-1.000000,0.970202,0.995282,-1.000000,-1.000000,0.985507,0.981752,0.961136,...,0.968089,1.001647,0.994140,0.988158,0.992660,-1.000000,0.956552,0.988887,0.981082,0.991524
2012-01-09,-1.000000,-1.000000,-1.000000,0.989911,0.991911,-1.000000,-1.000000,0.988678,0.984672,0.965122,...,0.975720,1.004078,0.992381,0.984631,0.996426,-1.000000,0.963793,0.985468,0.981905,0.987102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,1.747568,1.665700,2.513750,0.444158,0.542965,4.822593,1.321187,1.925682,2.012551,2.583405,...,1.039195,5.715043,1.058429,0.235828,1.845788,3.316001,2.592833,2.622808,2.215808,1.221669
2025-03-25,1.752316,1.671303,2.518000,0.440404,0.538741,4.795784,1.323292,1.934620,2.022247,2.607181,...,1.060701,5.728781,1.058311,0.235639,1.851925,3.320965,2.608376,2.593379,2.227675,1.201216
2025-03-26,1.728301,1.664899,2.454250,0.435476,0.541813,4.729206,1.311598,1.926495,2.006248,2.551478,...,1.058966,5.660389,1.051354,0.237528,1.863226,3.300044,2.571506,2.603497,2.207501,1.200663


# 7. Create rolling windows

In [19]:
def create_X_y(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data) - 1):
        window = data.iloc[i-window_size:i]
        target = data.iloc[i+1]
        if (window.values < 0).any() or (target.values < 0).any():
            continue
        X.append(window.values.flatten()) 
        y.append(target.values) 
    return np.array(X), np.array(y)

window_size = 30
X, y = create_X_y(normalized_close, window_size)
print(X.shape)
print(y.shape)
display(X)

(137, 990)
(137, 33)


array([[1.91432464, 2.27728202, 2.96800003, ..., 2.68511859, 2.13386786,
        1.22756591],
       [1.89715745, 2.26077439, 2.89249992, ..., 2.68637796, 2.1542377 ,
        1.22627607],
       [1.90173545, 2.26715249, 2.94799995, ..., 2.72867451, 2.15397334,
        1.22830299],
       ...,
       [1.75083829, 1.72963212, 2.40475006, ..., 2.72747359, 2.14811545,
        1.07278424],
       [1.74061447, 1.65809125, 2.40699997, ..., 2.76544971, 2.15797321,
        1.05951722],
       [1.75038162, 1.62133251, 2.42662506, ..., 2.79435686, 2.14753557,
        1.06006999]])

In [20]:
dataset_size = len(X)
print(f"dataset_size: {dataset_size}")
train_size = int(dataset_size * 0.8)
test_size = 30
val_size = dataset_size - train_size  - test_size

print(f"train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")
print(f"dataset_size: {dataset_size}, window_size: {window_size}")

X_train, X_val, X_test = X[:train_size], X[train_size: train_size + val_size], X[train_size + val_size: ]
y_train, y_val, y_test= y[:train_size], y[train_size: train_size + val_size], y[train_size + val_size: ]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


dataset_size: 137
train_size: 109, val_size: -2, test_size: 30
dataset_size: 137, window_size: 30


((109, 990), (109, 33), (0, 990), (0, 33), (30, 990), (30, 33))

In [21]:
rf = MultiOutputRegressor(
    RandomForestRegressor(
        random_state=42,
        criterion='squared_error',
        n_jobs=-1,
    )
    )
rf.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
Y_pred = rf.predict(X_test)
# display(pd.DataFrame(Y_pred, columns=used_data.columns))
Y_pred_mean = Y_pred.mean(axis=0)
# display(pd.DataFrame(Y_pred_mean, index=used_data.columns, columns=['Predicted Mean']))
y_pred_price = Y_pred_mean * first_prices.values
y_pred_df = pd.DataFrame(y_pred_price, index=used_data.columns)
display(y_pred_df.T)

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
0,38.138716,10.212258,97.65028,17.90719,13.922287,122.414848,59.379092,25.50288,40.268049,32.397166,...,29.122153,567.378748,94.990328,70.568767,118.153607,136.917504,68.003782,94.25533,74.352657,7.397753


In [30]:
used_data.columns

Index(['AIEQ', 'ASHR.L', 'BCHN.L', 'CORN', 'DBO', 'EDEN', 'EMXC', 'EWA', 'EWC',
       'EWG', 'EWH', 'EWJ', 'EWQ', 'EWS', 'EWU', 'EWW', 'GLD', 'IDUP.L',
       'IEMG', 'INDA', 'KSA', 'PSCC', 'SEMI.AS', 'SLV', 'SPY', 'TLT', 'USO',
       'VDE', 'VDNR.L', 'VGK', 'VNQ', 'VPL', 'XMAF.L'],
      dtype='object', name='Ticker')

In [29]:
predicted_prices = pd.DataFrame(Y_pred * first_prices.values, index=used_data.columns)
display(predicted_prices.T)

ValueError: Shape of passed values is (30, 33), indices imply (33, 33)

In [ ]:
yesterday_prices = used_data.iloc[-2]  # second last row 

today_prices_actual = used_data.iloc[-1] # last row

today_prices_predicted = pd.Series(y_pred_price.flatten(), index=used_data.columns)


comparison_df = pd.DataFrame({
    'Yesterday_actual': yesterday_prices,
    'Today_actual': today_prices_actual,
    'Today_pred': today_prices_predicted,
})

tickers_names = used_data.columns

correct_list = []
for ticker in tickers_names:
    yesterday = yesterday_prices[ticker]
    actual_today = today_prices_actual[ticker]
    predicted = today_prices_predicted[ticker]

    sign_actual = np.sign(actual_today - yesterday)
    sign_pred = np.sign(predicted - yesterday)

    if sign_actual == sign_pred:
        correct_list.append(1)
    else:
        correct_list.append(0)
comparison_df['Correct'] = correct_list

count_of_ones = correct_list.count(1)
print(f"Count of correct predictions: {count_of_ones}")
print(f"accuracy: {count_of_ones / len(correct_list)}")
display(comparison_df)
save_df = comparison_df.copy()
save_df['Yesterday actual'] = save_df['Yesterday_actual'].apply(lambda x: round(x, 2))
save_df['Today actual'] = save_df['Today_actual'].apply(lambda x: round(x, 2))
save_df['Today prediction'] = save_df['Today_pred'].apply(lambda x: round(x, 2))
save_df['Correct'] = save_df['Correct'].apply(lambda x: round(x, 2))
save_df.to_csv('rf_regression.csv')


Count of correct predictions: 19
accuracy: 0.5757575757575758


,Yesterday_actual,Today_actual,Today_pred,Correct
Ticker,,,,
AIEQ,37.658001,36.845001,38.138716,0
ASHR.L,10.460000,10.340000,10.212258,1
BCHN.L,97.080002,92.714996,97.650280,0
CORN,18.510000,18.459999,17.907190,1
DBO,14.130000,13.970000,13.922287,1
EDEN,105.330002,105.169998,122.414848,0
EMXC,56.110001,55.180000,59.379092,0
EWA,23.750000,23.469999,25.502880,0
EWC,41.250000,40.570000,40.268049,1
